In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from os.path import join as opj
from omegaconf import OmegaConf
from importlib import import_module
import argparse

import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader

from cldm.plms_hacked import PLMSSampler
from cldm.model import create_model
from utils import tensor2img


In [ ]:
def build_args():
    class Conf:
        config_path: str = "configs/VITON512.yaml"
        model_load_path: str = None
        batch_size: int = 2
        data_root_dir: str = "./data/zalando-hd-resized"
        repaint: bool = False
        unpair: bool = False
        save_dir: str = "./samples"
        denoise_steps: int = 50
        img_H: int = 256
        img_W: int = 192
        eta: float = 0.0
        
    return Conf()

args = build_args()

In [ ]:
batch_size = args.batch_size
img_H = args.img_H
img_W = args.img_W

config = OmegaConf.load(args.config_path)
config.model.params.img_H = args.img_H
config.model.params.img_W = args.img_W
params = config.model.params

print("params", params)

In [ ]:
model = create_model(config_path=None, config=config)
if args.model_load_path is not None:
    model.load_state_dict(torch.load(args.model_load_path, map_location="cpu"))
model = model.cuda()
model.eval()

In [ ]:
sum(p.numel() for p in model.parameters())

In [ ]:
sum(p.numel() for p in model.model.parameters())

In [ ]:
sum(p.numel() for p in model.first_stage_model.parameters())

In [ ]:
sum(p.numel() for p in model.cond_stage_model.parameters())

In [ ]:
sum(p.numel() for p in model.control_model.parameters())

In [ ]:
sampler = PLMSSampler(model)
dataset = getattr(import_module("dataset"), config.dataset_name)(
    data_root_dir=args.data_root_dir,
    img_H=img_H,
    img_W=img_W,
    is_paired=not args.unpair,
    is_test=True,
    is_sorted=True
)
dataloader = DataLoader(dataset, num_workers=4, shuffle=False, batch_size=batch_size, pin_memory=True)

shape = (4, img_H//8, img_W//8)
print("shape", shape)

In [ ]:
save_dir = opj(args.save_dir, "unpair" if args.unpair else "pair")
os.makedirs(save_dir, exist_ok=True)
for batch_idx, batch in enumerate(dataloader):
    print(f"{batch_idx}/{len(dataloader)}")
    z, c = model.get_input(batch, params.first_stage_key)
    bs = z.shape[0]
    c_crossattn = c["c_crossattn"][0][:bs]
    if c_crossattn.ndim == 4:
        c_crossattn = model.get_learned_conditioning(c_crossattn)
        c["c_crossattn"] = [c_crossattn]
    uc_cross = model.get_unconditional_conditioning(bs)
    uc_full = {"c_concat": c["c_concat"], "c_crossattn": [uc_cross]}
    uc_full["first_stage_cond"] = c["first_stage_cond"]
    for k, v in batch.items():
        if isinstance(v, torch.Tensor):
            batch[k] = v.cuda()
    sampler.model.batch = batch

    ts = torch.full((1,), 999, device=z.device, dtype=torch.long)
    start_code = model.q_sample(z, ts)     

    samples, _, _ = sampler.sample(
        args.denoise_steps,
        bs,
        shape, 
        c,
        x_T=start_code,
        verbose=False,
        eta=args.eta,
        unconditional_conditioning=uc_full,
    )

    x_samples = model.decode_first_stage(samples)
    for sample_idx, (x_sample, fn,  cloth_fn) in enumerate(zip(x_samples, batch['img_fn'], batch["cloth_fn"])):
        x_sample_img = tensor2img(x_sample)  # [0, 255]
        if args.repaint:
            repaint_agn_img = np.uint8((batch["image"][sample_idx].cpu().numpy()+1)/2 * 255)   # [0,255]
            repaint_agn_mask_img = batch["agn_mask"][sample_idx].cpu().numpy()  # 0 or 1
            x_sample_img = repaint_agn_img * repaint_agn_mask_img + x_sample_img * (1-repaint_agn_mask_img)
            x_sample_img = np.uint8(x_sample_img)

        to_path = opj(save_dir, f"{fn.split('.')[0]}_{cloth_fn.split('.')[0]}.jpg")
        cv2.imwrite(to_path, x_sample_img[:,:,::-1])